### Z-критерий для двух долей 

In [87]:
import pandas as pd
import numpy as np

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

Предположим, что мы хотим рекламировать некоторую услугу или товар с помощью рекламного баннера. Для этого у нас уже есть некоторый стандартный баннер, но дизайнеры разработали для нас новый, более прекрасный баннер. Нам с вами хочется проверить, правда ли, что новый баннер действительно лучше, чем старый, и нам имеет смысл заменить старый баннер на новый. Мы собрали 2 группы разных пользователей, по 1000 в каждой.

In [7]:
# Загрузка данных
data = pd.read_csv('banner_click_stat.txt', sep = '\t')
data.columns = ['banner_a', 'banner_b']
data.head()

,banner_a,banner_b
0,1,1
1,0,0
2,0,0
3,0,0
4,0,0


In [13]:
data.describe()

,banner_a,banner_b
count,999.000000,999.000000
mean,0.037037,0.053053
std,0.188947,0.224252
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


Среднее значение для ```banner_b``` больше, возможно новый баннер и вправду красивый и больше нравится пользователям.

Самое первое и самое простое, что мы можем сделать, это построить интервальную оценку для доли. То есть посмотреть, в каком диапазоне эта доля изменяется.

### Интервальная оценка для доли 
$$\hat{p}\pm z_{1-\frac{\alpha}{2}} \sqrt{\frac{\hat{p}\left(1-\hat{p}\right)}{n}}$$

Построим доверительные интервалы для 2-х баннеров и попробуем ответить на наш вопрос.

In [15]:
conf_interval_banner_a = proportion_confint(sum(data['banner_a']), data.shape[0], method = 'normal')
conf_interval_banner_b = proportion_confint(sum(data['banner_b']), data.shape[0], method = 'normal')

print('95% Confidence Interval (Banner A): ', conf_interval_banner_a)
print('95% Confidence Interval (Banner B): ', conf_interval_banner_b)

95% Confidence Interval (Banner A):  (0.02532619139352041, 0.04874788268055366)
95% Confidence Interval (Banner B):  (0.03915405923744482, 0.06695204686866128)


Интервалы пересекаются, значит однозначных выводов о том, что новый баннер лучше мы сделать не можем.

Попробуем оценить разность 2-х долей. Это можно сделать 2-мя способами:
- Построить доверительный интервал на разность 2-х долей (формула ниже)
- Воспользоваться Z - критерием для доли и проверить гипотезу о том, что доли разные

### Z-критерий для разности долей (независимые выборки)

$$\text{Доверительный интервал для разности 2-х долей:} p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

In [16]:
# Функция для построения доверительного интервала для разности 2-х долей
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [18]:
# Функция для расчета Z - статистики
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1/n1 + 1/n2))

In [75]:
# Функция для расчета p - value
def calculate_p_value(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [32]:
prop_conf_int_ind = proportions_diff_confint_ind(data['banner_a'], data['banner_b'], alpha = 0.05)
print('95% Confidence Interval For Proportion Difference: ', prop_conf_int_ind)

95% Confidence Interval For Proportion Difference:  (-0.03419088698591806, 0.0021588549538860274)


Ноль входит, возможно различий нет. Взглянем на ```p-value```

In [33]:
z_score_ind = proportions_diff_z_stat_ind(data['banner_a'], data['banner_b'])
p_value = calculate_p_value(z_score_ind, alternative = 'two-sided')
print('Z-Score For Proportion (independent Samples): ', z_score_ind)
print('P-Value: ', p_value)

Z-Score For Proportion (independent Samples):  -1.7258668408081945
P-Value:  0.08437137142549567


На двусторонней альтернативе мы не можем отвергнуть нулевую гипотезу.

Поробуем сипользовать одностороннюю альтернативу ```less``` (т.е. нулевой гипотезой будет: новый баннер хуже)

In [34]:
p_value = calculate_p_value(z_score_ind, alternative = 'less')
print('P-Value: ', p_value)

P-Value:  0.042185685712747785


Нулевая гипотеза отвергается, следовательно, баннер явно не хуже прежнего, возможно даже лучше.

Теперь предположим, что пользователи были одни и теже (т.е. группы становятся связанными). Мы также можем использовать пердыдущие методы, но с поправкой на зависимость.

Для расчета доверительного интервала и ```Z-статистики```, необходимо воспользоваться **таблицей сопряженности** и следующими формулами:

### Z-критерий для разности долей (зависимые выборки)

$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$

In [69]:
# Функция для расчета доверительного интервала
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha/2)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [72]:
# Функция расчета Z - статистики
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [37]:
prop_conf_int_rel = proportions_diff_confint_rel(data['banner_a'], data['banner_b'], alpha = 0.05)
print('95% Confidence Interval For Proportion Difference: ', prop_conf_int_rel)

95% Confidence Interval For Proportion Difference:  (-0.02671593324626175, -0.0053160987857702804)


Ноль теперь не входит, возиожно различия есть и баннер лучше. Взглянем на ```p-value```

In [39]:
z_score_rel = proportions_diff_z_stat_rel(data['banner_a'], data['banner_b'])
p_value = calculate_p_value(z_score_rel, alternative = 'two-sided')
print('Z-Score For Proportion (Dependent Samples): ', z_score_rel)
print('P-Value: ', p_value)

Z-Score For Proportion (Dependent Samples):  -2.9337436815375386
P-Value:  0.0033490064943413334


Проверим также одностороннюю альтернативу

In [41]:
p_value = calculate_p_value(z_score_rel, alternative = 'less')
print('P-Value: ', p_value)

P-Value:  0.0016745032471706873


```P-value``` еще меньше, т.е. мы еще более увереннее можем отвергнуть нулевую гипотезу

### Задания из теста

**1)** В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате. 

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён. 

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [95]:
# Опишем эксперимент 
n_samples_a = 34
n_samples_b = 16

# Доли успехов в 2-х независимых группах
p_a = 10 / n_samples_a
p_b = 4 / n_samples_b

Воспользуемся Z - критерием для 2-х долей (выборки независимые)

In [96]:
# Не будем вводить функцию, а просто посчтитаем
P = ((p_a*n_samples_a) + (p_b*n_samples_b))/(n_samples_a + n_samples_b)
z_score = (p_a - p_b)/np.sqrt(P*(1-P)*((1/n_samples_a) + (1/n_samples_b)))

In [97]:
# Достигаемый уровень значимости
p_value = calculate_p_value(z_score, alternative = 'greater')
print('P-Value: ', round(p_value,4))

P-Value:  0.3729


Значения между группами не отличаются значимо. Значит, по данному эксперименту, зевота не является заразительной

**2)** Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки.

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции ```sklearn.cross_validation.train_test_split```, зафиксируйте random ```state = 1```. На оставшихся 150 настройте два классификатора поддельности банкнот:

- логистическая регрессия по признакам х1, х2, х3
- логистическая регрессия по признакам x4, x5, x6 

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили ```5.5*10^-8``` нужно ввести -8.

In [98]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [99]:
# Загрузка данных
data = pd.read_csv('banknotes.txt', sep = '\t')
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [100]:
# Взглянем на балансы классов
data['real'].value_counts()

1    100
0    100
Name: real, dtype: int64

In [101]:
# Отделим матрицу признаков и таргет
x = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [102]:
# Создаем выборки согласно заданиям 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 50, shuffle = True, random_state = 1)

In [103]:
log_reg_model_1 = LogisticRegression(random_state = 1, n_jobs = -1)
log_reg_model_2 = LogisticRegression(random_state = 1, n_jobs = -1)

# Отберем необходимые признаки согласно заданию (х1, х2, х3)
x_train_1, x_test_1 = x_train.iloc[:, :3], x_test.iloc[:, :3]

# Отберем необходимые признаки согласно заданию (x4, x5, x6)
x_train_2, x_test_2 = x_train.iloc[:, 3:], x_test.iloc[:, 3:]

In [104]:
# Обучаем модели
log_reg_model_1.fit(x_train_1, y_train)
log_reg_model_2.fit(x_train_2, y_train)

# Предсказываем
log_reg_preds_1 = log_reg_model_1.predict(x_test_1)
log_reg_preds_2 = log_reg_model_2.predict(x_test_2)

# Доля верных ответов (точность) моделей
accuracy_1 = accuracy_score(y_test, log_reg_preds_1)
accuracy_2 = accuracy_score(y_test, log_reg_preds_2)

print('Accuracy (Model 1): ', accuracy_1)
print('Accuracy (Model 2): ', accuracy_2)
print()
print('Error (Model 1): ', 1 - accuracy_1)
print('Error (Model 2): ', 1 - accuracy_2)

Accuracy (Model 1):  0.8
Accuracy (Model 2):  0.98

Error (Model 1):  0.19999999999999996
Error (Model 2):  0.020000000000000018


Видно, что ошибки 2-х классификаторов существенно отличаются. Но значимы ли эти отличия?

Сформулируем ряд гипотез:

$H_0\colon$ различий между 2-мя классификаторами нет

$H_1\colon$ различия есть

In [105]:
# Сформулируем доли верных ответов для 2-х моделей
error_portion_1 = [1 if log_reg_preds_1[indx] == y_test.values[indx] else 0 for indx in range(len(log_reg_preds_1))]
error_portion_2 = [1 if log_reg_preds_2[indx] == y_test.values[indx] else 0 for indx in range(len(log_reg_preds_2))]

In [107]:
# Расчитаем Z - статистику
z_score = proportions_diff_z_stat_rel(error_portion_1, error_portion_2)
p_val = calculate_p_value(z_score, alternative = 'two-sided')
print('p-value: ', round(p_val, 4))

p-value:  0.0009


Обнаружены значимые различия. Качество действительно отличается.

**3)** В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [83]:
# Оценим 95% Доверительный интервал для зависимых выборок (функция proportions_diff_confint_rel)
conf_interval = proportions_diff_confint_rel(error_portion_1, error_portion_2)
print(f'95% Confidence Interval (Dependent Samples): [{round(conf_interval[0], 4)}, {round(conf_interval[1], 4)}]' )

95% Confidence Interval (Dependent Samples): [-0.2865, -0.0735]


Ноль не входит, значит различия статистически значимы (```p-value``` также свидетельствует об этом из предыдузей задачи)

**4)** Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов. 

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

In [84]:
# Опишем ГС
n_samples = 200000
population_mean = 525
sigma = 100

# Опишем Выборку
n = 100
sample_mean = 541.4

$H_0\colon$ программа неэффективна

$H_1\colon$ программа работает

В этой задаче мы имеем дело с непрерывной СВ, следовательно доли мы не рассматриваем. Так-как нам известно СКО ГС, то будем использовать одновыборочный Z - критерий.

In [85]:
# Рассчитаем Z - статистику 
se = sigma/np.sqrt(n)
z_score = (sample_mean - population_mean)/se
print('Z - Score: ', z_score)

Z - Score:  1.6399999999999977


In [91]:
# Рассчитаем p - value (по условию, односторонняя альтернатива)
p_val = 1 - stats.norm.cdf(z_score)
print('p-value: ', round(p_val, 4))

p-value:  0.0505


К сожалению, отвергнуть нулевую гипотезу мы не можем. Скорее всего программа неэффективна.

**5)** Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

In [92]:
new_sample_mean = 541.5

se = sigma/np.sqrt(n)
z_score = (new_sample_mean - population_mean)/se
print('Z - Score: ', z_score)

Z - Score:  1.65


In [93]:
p_val = 1 - stats.norm.cdf(z_score)
print('p-value: ', round(p_val, 4))

p-value:  0.0495


Новое значение, позволяет отвергнуть нулевую гипотезу